# Импортируем аудиозапись

In [1]:
from pydub import AudioSegment
import numpy as np

song = AudioSegment.from_mp3("raw/horse.mp3")

print(f'{song.dBFS = }')
print(f'{song.channels = }')
print(f'{song.sample_width = }')
print(f'{song.frame_rate = }')
print(f'{song.frame_width = }')
print(f'{song.rms = }')
print(f'{song.max = }')
print(f'{song.max_dBFS = }')
print(f'{song.duration_seconds = }')
print(f'{song.frame_count() = }')

left, right = (song).split_to_mono()

left_data = np.array(left.get_array_of_samples()).T.astype(np.float32)
right_data = np.array(right.get_array_of_samples()).T.astype(np.float32)

left_data = left_data[:300_000]
right_data = right_data[:300_000]

print("Frames count on 1 channel: ", len(left_data))
print("Frames count on 2 channel: ", len(right_data))

song.dBFS = -17.14877076605211
song.channels = 2
song.sample_width = 2
song.frame_rate = 44100
song.frame_width = 4
song.rms = 4550
song.max = 26212
song.max_dBFS = -1.9389955135123373
song.duration_seconds = 217.42666666666668
song.frame_count() = 9588516.0
Frames count on 1 channel:  300000
Frames count on 2 channel:  300000


# Преобразуем в influx line protocol format

In [2]:
import time
write_data = []
for i, [l, r] in enumerate(zip(left_data, right_data)):
    json_body = {
        "measurement": "a",
        "time": int(time.time() * 1_000_000_000),
        "tags": {
            "a": "i",
        },
        "fields": {
            "l": float(l),
            "r": float(r),
        }
    }
    write_data.append(json_body)

# Запись в influxDB

In [4]:
import influxdb_client

bucket_in = "bucket_in"
bucket_out = "bucket_out"

org = "mipt"
token='AxZQBiPLqGo7nkwQxuga16V_Ui8qY8livmN-TGKiqtTO3LOfAd3KNRslGYs4Eorcml1IgYlnwb8YtzM1jX22-Q=='

# Store the URL of your InfluxDB instance
url="http://localhost:8086"

client = influxdb_client.InfluxDBClient(
    url=url,
    token=token,
    org=org
)

with client.write_api() as write_api:
    write_api.write(bucket=bucket_in, org=org, record=write_data)

# Ждем пока task в influxDB обработает аудио

In [ ]:
time.sleep(120)

# Чтение данных

In [10]:
import json
bucket = bucket_out

start = time.time()
left_data  = client.query_api().query(f'from(bucket:"{bucket}") |> range(start: -10m) |> filter(fn: (r) => r["_field"] == "l")')
right_data = client.query_api().query(f'from(bucket:"{bucket}") |> range(start: -10m) |> filter(fn: (r) => r["_field"] == "r")')
print('query time:', time.time() - start)
start = time.time()

left_vals =  [min(max(-32768, int(point['_value'])), 32767) for point in json.loads(left_data.to_json())]
right_vals = [min(max(-32768, int(point['_value'])), 32767) for point in json.loads(right_data.to_json())]
print('analyze time:', time.time() - start)

print(f'{len(left_vals) = }')
print(f'{len(right_vals) = }')

query time: 0.011051416397094727
analyze time: 0.00027179718017578125
len(left_vals) = 0
len(right_vals) = 0


# Преобразование данных в аудио

In [7]:
import array
import io
import scipy.io.wavfile

left_vals  = np.array(left_vals)
right_vals = np.array(right_vals)

left_arr = array.array(song.array_type, left_vals)
right_arr = array.array(song.array_type, right_vals)

wav_io = io.BytesIO()
scipy.io.wavfile.write(wav_io, 44100, np.array(left_arr).T)
wav_io.seek(0)
new_song = AudioSegment.from_wav(wav_io)
file_handle = new_song.export("raw/horse_out.mp3", format="mp3")